# BLAS and LAPACK

"BLAS is a collection of low-level matrix and vector arithmetic operations (“multiply a vector by a scalar”, “multiply two matrices and add to a third matrix”, etc ...).

LAPACK is a collection of higher-level linear algebra operations. Things like matrix factorizations (LU, LLt, QR, SVD, Schur, etc) that are used to do things like “find the eigenvalues of a matrix”, or “find the singular values of a matrix”, or “solve a linear system”. LAPACK is built on top of the BLAS; many users of LAPACK only use the LAPACK interfaces and never need to be aware of the BLAS at all. LAPACK is generally compiled separately from the BLAS, and can use whatever highly-optimized BLAS implementation you have available."

from SO answer here https://stackoverflow.com/questions/17858104/what-is-the-relation-between-blas-lapack-and-atlas\

docs from the intel website https://software.intel.com/en-us/mkl-developer-reference-c

In [90]:
import torch
import numpy as np

## Matrix and vector algebra 

In [126]:
# regular matrix multiplication
mat1 = torch.ones(2,10)
mat2 = torch.ones(10,3)
res = torch.mm(mat1, mat2)
print(f'mm : {res}')

# multipling a batch of matrices
M = torch.ones(2,3)*100
bmat1 = torch.ones(4,2,10)
bmat2 = torch.ones(4,10,3)
res = torch.bmm(bmat1, bmat2)
print(f'bmm : {res}')

# multipling and adding together a batch of matrices then add a matrix to the end result
M = torch.ones(2,3)*100
bmat1 = torch.ones(4,2,10)
bmat2 = torch.ones(4,10,3)
res = torch.addbmm(M, bmat1, bmat2, alpha=1, beta=0.5)
print(f'addbmm : {res}')

# other variants are present like: torch.addmm, torch.baddbmm,  ..

mm : tensor([[10., 10., 10.],
        [10., 10., 10.]])
bmm : tensor([[[10., 10., 10.],
         [10., 10., 10.]],

        [[10., 10., 10.],
         [10., 10., 10.]],

        [[10., 10., 10.],
         [10., 10., 10.]],

        [[10., 10., 10.],
         [10., 10., 10.]]])
addbmm : tensor([[90., 90., 90.],
        [90., 90., 90.]])


In [127]:
# multiply a matrix and a vector
mat = torch.randn(2, 3)
vec = torch.ones(3)
res = torch.mv(mat, vec)
print(f'mv : {res}')

# multiply a matrix and a vector and add another vector
V = torch.ones(2)*10
mat = torch.randn(2, 3)
vec = torch.ones(3)
res = torch.addmv(V, mat, vec)
print(f'addmv : {res}')

mv : tensor([ 0.5911, -1.8514])
addmv : tensor([8.6583, 8.2520])


In [138]:
# matmul: multiplying tensors with any dimension (generalization)
"""
The behavior depends on the dimensionality of the tensors as follows:

If both tensors are 1-dimensional, the dot product (scalar) is returned.
If both arguments are 2-dimensional, the matrix-matrix product is returned.
If the first argument is 1-dimensional and the second argument is 2-dimensional, 
    a 1 is prepended to its dimension for the purpose of the matrix multiply. 
    After the matrix multiply, the prepended dimension is removed.
If the first argument is 2-dimensional and the second argument is 1-dimensional,
    the matrix-vector product is returned.
If both arguments are at least 1-dimensional and at least one argument is N-dimensional (where N > 2), 
    then a batched matrix multiply is returned. If the first argument is 1-dimensional, 
    a 1 is prepended to its dimension for the purpose of the batched matrix multiply and removed after. 
    If the second argument is 1-dimensional, a 1 is appended to its dimension for the purpose of the batched matrix multiple and removed after. The non-matrix (i.e. batch) dimensions are broadcasted (and thus must be broadcastable). For example, if tensor1 is a (j \times 1 \times n \times m)(j×1×n×m) tensor and tensor2 is a (k \times m \times p)(k×m×p) tensor, out will be an (j \times k \times n \times p)(j×k×n×p) tensor.
"""
tensor1 = torch.randn(2,3)
tensor2 = torch.randn(3,2)
print(f'matrix*matrix :{torch.matmul(tensor1, tensor2)}', end='\n\n')
tensor1 = torch.randn(3)
tensor2 = torch.randn(3,2)
print(f'vector*matrix :{torch.matmul(tensor1, tensor2)}', end='\n\n')
tensor1 = torch.randn(3,2)
tensor2 = torch.randn(2)
print(f'matrix*vector :{torch.matmul(tensor1, tensor2)}', end='\n\n')
tensor1 = torch.randn(3)
tensor2 = torch.randn(3)
print(f'vector*vector :{torch.matmul(tensor1, tensor2)}', end='\n\n')
tensor1 = torch.randn(5,2,3)
tensor2 = torch.randn(3)
print(f'3Dtensor*vector :{torch.matmul(tensor1, tensor2)}', end='\n\n') # also the other way around works

matrix*matrix :tensor([[-0.2968, -2.0897],
        [ 1.0259,  0.2704]])

vector*matrix :tensor([1.2198, 0.2322])

matrix*vector :tensor([3.9841, 0.6078, 1.2510])

vector*vector :-0.6373900771141052

3Dtensor*vector :tensor([[-0.2895, -0.6748],
        [-0.6835,  0.6867],
        [ 0.3335,  0.9583],
        [ 0.1476, -0.2026],
        [-0.2862,  0.3676]])



In [88]:
# chain matrix multuplication
mat1 = torch.randn(2,3)
mat2 = torch.randn(3,4)
mat3 = torch.randn(4,2)
res = torch.chain_matmul(mat1, mat2, mat3)
print(res)

tensor([[ 0.7495,  0.6446],
        [-0.5327,  1.9050]])


In [185]:
# outer product
torch.ger(torch.Tensor([1,2]),torch.Tensor([3,4]))

tensor([[3., 4.],
        [6., 8.]])

In [ ]:
# other obvious function are present like: torch.matrix_power(input, n), torch.dot(v1,v2)

## Extracting matrix attributes (det, rank, eigenvalues..)

In [ ]:
torch.inverse
torch.det 
torch.eig # eigenvalues and eigenvectors
torch.matrix_rank
# ..

## Matrix decompositions


In [155]:
# singluar value decomposition
"""
U, S, V = torch.svd(A) returns the singular value decomposition of a real matrix A of size (n x m) 
such that A = USV^T.
U is of shape (n×n).
S is a diagonal matrix of shape (n×m), 
    represented as a vector of size min(n,m) containing the non-negative diagonal entries.
V is of shape (m×m)."""
A = torch.Tensor([[1,1,1],[0,1,0]])
U,S,V = torch.svd(A)
print(f'U = {U} \n S = {S} \n V = {V}')
print(f'A = USV^T = {torch.chain_matmul(U,torch.diag(S),torch.transpose(V,0,1))}')


U = tensor([[-0.9239, -0.3827],
        [-0.3827,  0.9239]]) 
 S = tensor([1.8478, 0.7654]) 
 V = tensor([[-0.5000, -0.5000],
        [-0.7071,  0.7071],
        [-0.5000, -0.5000]])
A = USV^T = tensor([[ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [-5.9605e-08,  1.0000e+00, -2.9802e-08]])


In [207]:
# Lower-triangular and Upper-triangular (LU) decomposition
A = torch.randn(2,5,5)
# computed packed decompositions and pivots
B = torch.btrifact(A)
# returns the unpacked versions, passes only the pivots
torch.btriunpack(A,B[1])

# 


(tensor([[[0., 1., 0., 0., 0.],
          [0., 0., 0., 1., 0.],
          [1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 1.],
          [0., 0., 1., 0., 0.]],
 
         [[0., 0., 0., 1., 0.],
          [0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 1.],
          [0., 1., 0., 0., 0.],
          [1., 0., 0., 0., 0.]]]),
 tensor([[[ 1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.5793,  1.0000,  0.0000,  0.0000,  0.0000],
          [-1.1238, -0.4372,  1.0000,  0.0000,  0.0000],
          [ 0.3117, -1.1657, -1.2497,  1.0000,  0.0000],
          [ 0.3741,  0.1693, -0.5634, -0.1194,  1.0000]],
 
         [[ 1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [-0.0652,  1.0000,  0.0000,  0.0000,  0.0000],
          [ 1.0614,  0.1640,  1.0000,  0.0000,  0.0000],
          [-0.7786,  1.5405,  0.8784,  1.0000,  0.0000],
          [-1.6803, -0.4199, -0.4098,  1.9121,  1.0000]]]),
 tensor([[[-0.7550,  2.2018,  1.3260,  0.4933,  0.9644],
          [ 0.0000, -0.5526,  0.0224, -0.

In [194]:
# QR decomposition

"""
Computes the QR decomposition of a matrix input, and returns matrices Q and R such that 
    input = Q R, with Q being an orthogonal matrix and R being an upper triangular matrix.
This returns the thin (reduced) QR factorization."""
A = torch.randn(4,3)
Q, R = torch.qr(A)
print(f'Q = {Q} \n R = {R}')

Q = tensor([[-0.0546,  0.0188,  0.0082],
        [ 0.3140, -0.1597,  0.9358],
        [-0.8903, -0.3889,  0.2330],
        [ 0.3252, -0.9072, -0.2644]]) 
 R = tensor([[-0.6418,  1.8647, -0.8099],
        [ 0.0000,  1.4626,  0.0718],
        [ 0.0000,  0.0000, -0.4858]])


In [ ]:
# other decompositions implemented: torch.cholesky(), ..

## Systems of linear equations


In [232]:
# the following functions solve systems of linear equations
"""
Batch LU solve.
Returns the LU solve of the linear system Ax = bAx=b."""
A = torch.randn(1,5,5)
# computed packed decompositions and pivots (used for the input of the solver)
LU_pack  = torch.btrifact(A)
# define the coefficient vector 
b = torch.Tensor([[1,2,3,4,5]])
print(f'solution = {torch.btrisolve(b, *LU_pack)[0]}')

# other variants on the same flavour

solution = tensor([-7.0601, 10.5317, -5.6973, -2.3638, -1.6744])


# Comparison operations

### common comparison operations

In [62]:
x = torch.tensor([[1,2,3]])
y = torch.tensor([[4,2,1]])
# min - max
print(f'torch.max(x) = {torch.max(x)}', end = ';  ') #tensor-wise
print(f'torch.max(x, y) = {torch.max(x, y)}', end = ';\n') #element-wise
print(f'torch.min(x) = {torch.min(x)}', end = ';  ') 
print(f'torch.min(x, y) = {torch.min(x, y)}', end = ';\n') 
# greater than (gt), greater equal (ge), lt, le, eq, not equal (ne), all element-wise if same shape or see below
# torch.equal returns bool if shape and elements are the same

torch.max(x) = 3;  torch.max(x, y) = tensor([[4, 2, 3]]);
torch.min(x) = 1;  torch.min(x, y) = tensor([[1, 2, 1]]);


In [67]:
### interesting properties due to numpy broadcasting semantics
x = torch.tensor([[1,2,3],
                  [-1,6,-3]])
y = torch.tensor([4,2,1])
print(f'torch.min(x) = {torch.min(x)}') 
print(f'torch.min(x, y) = {torch.min(x, y)}', end = ';\n') 
print(f'torch.min(y,x) = {torch.min(y,x)}', end = ';\n') 

torch.min(x) = -3
torch.min(x, y) = tensor([[[ 1,  2,  1],
         [-1,  2, -3]]]);
torch.min(y,x) = tensor([[[ 1,  2,  1],
         [-1,  2, -3]]]);


### allclose and sorting

In [241]:
### allclose between two tensor element wise
"""
∣self−other∣≤ atol + rtol×∣other∣
"""
self = torch.tensor([10,0.001])
other = torch.tensor([10.01, 0.011])
torch.allclose(self, other, rtol=1e-01, atol=1e-01, equal_nan=False)

True

In [272]:
### returns tuple of sorted tensor and sorting indices along a given dimension
a = torch.randn(2, 3)
print(a)
print(torch.sort(a, dim =1)) # sort each row


tensor([[-0.4570, -1.0176, -1.7433],
        [ 1.0485, -0.4385,  0.2557]])
(tensor([[-1.7433, -1.0176, -0.4570],
        [-0.4385,  0.2557,  1.0485]]), tensor([[2, 1, 0],
        [1, 2, 0]]))


### very useful comparison operations

In [281]:
tensor = torch.randn(3,4)
print(tensor)
# topk to find the largest value in a tensor (always used for categorical features)
print(f'topk = {torch.topk(tensor.t(), k = 2, dim=0, largest=True, sorted=True, out=None)}') # k is how many largest

# returns the smallest k 
print(f'kthvalue = {torch.kthvalue(tensor.t(), k=1, dim=1, keepdim=False, out=None)}') # k is the kth smallest

# note that dim works differently than in sort

tensor([[ 0.2802,  0.7261, -0.4381,  1.2156],
        [ 0.1427, -0.1381,  1.0498,  0.6352],
        [-0.7170, -0.1572, -0.4860,  0.6571]])
topk = (tensor([[ 1.2156,  1.0498,  0.6571],
        [ 0.7261,  0.6352, -0.1572]]), tensor([[3, 2, 3],
        [1, 3, 1]]))
kthvalue = (tensor([-0.7170, -0.1572, -0.4860,  0.6352]), tensor([2, 2, 2, 1]))


# Other operations

In [292]:
# almost all these operations are pretty self-explanatory but here I mention few of them that could be tricky 
#or deserve a bit of explanation

tensor = torch.randn(2,3,3)
print(tensor)
print()
print('roll the subtensors')
print(torch.roll(tensor, 1, dims = 0))
print()
print('roll the rows in the subtensors')
print(torch.roll(tensor, 1, dims = 1))
print()
print('roll the columns in subtensors')
print(torch.roll(tensor, 1, dims = 2))



tensor([[[ 1.4039e+00, -1.5218e-01,  4.9487e-01],
         [ 7.0787e-01, -2.2500e-01,  1.3149e-01],
         [-2.0341e-01,  8.5648e-01,  2.4924e-01]],

        [[ 2.1495e+00,  1.0697e-03,  4.5938e-01],
         [-8.6795e-01, -1.3056e-01,  6.0574e-01],
         [-1.4216e+00,  1.3023e+00,  7.8987e-01]]])

roll the subtensors
tensor([[[ 2.1495e+00,  1.0697e-03,  4.5938e-01],
         [-8.6795e-01, -1.3056e-01,  6.0574e-01],
         [-1.4216e+00,  1.3023e+00,  7.8987e-01]],

        [[ 1.4039e+00, -1.5218e-01,  4.9487e-01],
         [ 7.0787e-01, -2.2500e-01,  1.3149e-01],
         [-2.0341e-01,  8.5648e-01,  2.4924e-01]]])

roll the rows in the subtensors
tensor([[[-2.0341e-01,  8.5648e-01,  2.4924e-01],
         [ 1.4039e+00, -1.5218e-01,  4.9487e-01],
         [ 7.0787e-01, -2.2500e-01,  1.3149e-01]],

        [[-1.4216e+00,  1.3023e+00,  7.8987e-01],
         [ 2.1495e+00,  1.0697e-03,  4.5938e-01],
         [-8.6795e-01, -1.3056e-01,  6.0574e-01]]])

roll the columns in subtensors
te

# torch.Tensor class methods

In [301]:
# most of the functions from torch are implemented both as in-place and not in-place methods in the torch.Tensor class.

In [300]:
# inplace tensor version of map
def add_1000(x,y):
    return x + 1000
tensor1 = torch.randn(2,2,3)
print(tensor1)
tensor1.map_(tensor1,add_1000)
print(tensor1)

tensor([[[-0.3846,  1.5621,  0.1579],
         [-0.2362,  0.5538, -0.8171]],

        [[ 0.0506, -1.0614,  1.8333],
         [ 0.0219, -0.2927,  1.3855]]])
tensor([[[ 999.6154, 1001.5621, 1000.1579],
         [ 999.7638, 1000.5538,  999.1829]],

        [[1000.0506,  998.9386, 1001.8333],
         [1000.0219,  999.7073, 1001.3856]]])


In [306]:
# repeating a tensor (similar to numpy.tile)
"""
Signature: np.tile(A, reps)
Docstring:
Construct an array by repeating A the number of times given by reps.
Parameters
----------
A : array_like
    The input array.
reps : array_like
    The number of repetitions of `A` along each axis.
"""
x = torch.tensor([1, 2, 3])
print(x)
print(x.repeat(4, 2))
x.repeat(4, 2, 1).size()
torch.Size([4, 2, 3])

tensor([1, 2, 3])
tensor([[1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3]])


torch.Size([4, 2, 3])

In [308]:
np.tile?

on broadcasting semantics
https://pytorch.org/docs/stable/notes/broadcasting.html